In [1]:
import math
import numpy as np
from keras.datasets import mnist
import csv
import time
from numba import cuda
import os 
from os import listdir
from os.path import isfile, join
import cv2


train_path = r'C:\Users\user\Neural_Network\Training_data'


topic = ["idx", "train_y", "pred_y", "T/F"]
#scale_train = len(train_y)
#scale_valid = len(test_y)

In [2]:
label_folders = []
total_size = 0
#最外層資料夾(including n class folders)
def train_read(data_path):
    total_size = 0
    for root, dirts, files in os.walk(data_path):
        for dirt in dirts:
            label_folders.append(dirt)
        total_size += len(files)
        # n files
    # read datas and their classes
    base_x_train = []
    base_y_train = []

    for label_folder in label_folders:
        labelPath = data_path+r'\\'+ label_folder

        files = [f for f in listdir(labelPath) if isfile(join(labelPath, f))]

        for file in files:
            path = join(labelPath, file)

            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            base_x_train.append(img)
            base_y_train.append(int(label_folder))
    return base_x_train, base_y_train

def valid_read(data_path):
    base_x_train = []
    for file in listdir(data_path):
        path = os.path.join(data_path, file)
        base_x_train.append(path)
    return base_x_train


In [3]:
#train_X, train_y = train_read(train_path)
(train_X, train_y), (test_X, test_y) = mnist.load_data()
scale_train = len(train_X)
scale_valid = len(train_y)

In [12]:
valid_path = "./BP_valid1.csv"
txt_path = "./410985015.txt"
train_path = './BP_train1.csv'

def sigmoid(x):
    if x >= 0:
        return 1.0/(1.0 + np.exp(-x)) 
    else:
        return np.exp(x)/(1.0 + np.exp(x)) 

class BP(object):
    
    def __init__(self, input_size, fc_layers = [512,64,10]):
        self.weights = []
        in_channels = input_size
        for i in range(len(fc_layers)):
            self.weights.append(np.zeros(shape=(in_channels, fc_layers[i])))
            #self.weights.append(np.random.normal(size=(in_channels, fc_layers[i])))
            in_channels = fc_layers[i]
    
    def Forward_pass(self, x, weights):
        res = np.dot(x, weights)
        res = 1.0/(1.0 + np.exp(-res)) 
        return res
   
    def Backward_pass(self, lr, sigma, layer_output, w):
        sigma_ex = sigma[None, :]
        output_ex = layer_output[:, None]
        w += np.matmul(output_ex, sigma_ex) * lr
        return w
        
    def train(self, data, targets, epoches, lr, idx=1):
        acc = 0
        for epoch in range(epoches):
            print("執行開始".center(10//2, '-'))
            start = time.perf_counter()
            true_num = 0
            idx = 1
            for r, t in zip(data, targets):
                # with open(train_path, 'a', encoding='UTF8', newline='') as f:
                #    writer = csv.writer(f)
                #    writer.writerow(self.weights[0].flatten())
                ratio = (idx / scale_train)
                movedBar = '*' * int(10*ratio)
                residualBar = '.' * int(10*(1 - ratio))
                dur = time.perf_counter() - start
                print("\r{:3.0f}%[{}->{}]{:.2f}s acc:{:.2f}% ".format(ratio*100, movedBar, residualBar, dur, (true_num/idx)*100), end='')
                
                #r= r/255
                r = r.flatten()
                target = np.zeros(10)
                target[t] = 1
                fc_output = []
                fc_in = r
                for i in range(len(self.weights)):
                    fc_output.append(self.Forward_pass(fc_in, self.weights[i]))
                    fc_in = fc_output[-1]
                tmp = [idx, t, fc_output[-1].argmax(), 1 if t == fc_output[-1].argmax() else 0]
                idx+=1
                loss = target - fc_output[-1]
                
                if(t == fc_output[-1].argmax()):
                    true_num+=1
                sigma = []
                coefficient = loss
                for i in range(len(fc_output)):
                    sigma.append(coefficient*fc_output[-1-i]*(1-fc_output[-1-i]))
                    coefficient = np.dot(self.weights[-1-i], sigma[-1])
                    #coefficient = np.matmul(sigma[-1], self.weights[-1-i].T)
                coefficient_a = r
                for i in range(len(self.weights)):
                    self.weights[i] = self.Backward_pass(lr, sigma[-1-i], coefficient_a, self.weights[i])
                    coefficient_a = fc_output[i]
                acc = (true_num/idx)*100
            print("\n"+"執行結束".center(10//2, '-'))
            print("")
            break
        return acc
    
    def predict(self, data, targets, idx=1):
        with open(valid_path, 'a', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(topic)
        print("執行開始".center(10//2, '-'))
        start = time.perf_counter()
        true_num = 0
        idx = 1
        acc = 0
        for r, t in zip(data, targets):
            #ratio = (idx / scale_valid)
            #movedBar = '*' * int(10*ratio)
            #residualBar = '.' * int(10*(1 - ratio))
            #dur = time.perf_counter() - start
            #print("\r{:3.0f}%[{}->{}]{:.2f}s acc:{:.2f}% ".format(ratio*100, movedBar, residualBar, dur, (true_num/idx)*100), end='')
            
            r = r.flatten()
            #r = r/255
            fc_output = []
            fc_in = r
            
            for i in range(len(self.weights)):
                fc_output.append(self.Forward_pass(fc_in, self.weights[i]))
                fc_in = fc_output[-1]
                
            tmp = [idx, t, fc_output[-1].argmax(), 1 if t == fc_output[-1].argmax() else 0]
            if t == fc_output[-1].argmax(): 
                true_num+=1
            with open(valid_path, 'a', encoding='UTF8', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(tmp)
            idx+=1
            acc = (true_num/idx)*100
        print("acc:{:.2f}% ".format((true_num/idx)*100))    
        print("執行結束".center(10//2, '-'))
        print("")
        return acc
    
    def test(self, data, idx=1):
        print("執行開始".center(10//2, '-'))

        for r in data:
            
            arr = cv2.imread(r, cv2.IMREAD_GRAYSCALE)
            arr = arr.flatten()
            arr = arr/255
            fc_output = []
            fc_in = arr
            
            for i in range(len(self.weights)):
                fc_output.append(self.Forward_pass(fc_in, self.weights[i]))
                fc_in = fc_output[-1]
                
            
            with open(txt_path, 'a') as f:
                f.write(os.path.splitext(os.path.basename(r))[0]+" "+str(fc_output[-1].argmax())+'\n')
            idx+=1
                
        print("\n"+"執行結束".center(10//2, '-'))
        print("")
        return
    
    def print_weights(self):
        for i in range(len(self.weights)):
            print(self.weights[i].shape)

In [13]:
model = BP(28*28)

In [14]:
for i in range (3):
    #train_X = np.array(train_X)
    #train_y = np.array(train_y)
    indices = np.arange(train_X.shape[0])
    np.random.shuffle(indices)
    train_X = train_X[indices]
    train_y = train_y[indices]
    model.train(data=train_X, targets=train_y, epoches=1, lr=0.1)
    model.predict(data=test_X, targets=test_y)
    
    # indices = np.arange(train_X.shape[0])
    # np.random.shuffle(indices)
    # train_X = train_X[indices]
    # train_y = train_y[indices]
    # t_w = model.train(data=train_X, targets=train_y, epoches=1, lr=0.1)
    # v_w = model.predict(data=test_X, targets=test_y)
    

-執行開始
100%[**********->]250.58s acc:10.46% 
-執行結束

-執行開始
acc:11.35% 
-執行結束

-執行開始
100%[**********->]253.35s acc:10.79% 
-執行結束

-執行開始
acc:10.28% 
-執行結束

-執行開始
100%[**********->]254.47s acc:10.61% 
-執行結束

-執行開始
acc:11.35% 
-執行結束



In [7]:
path=r'C:\Users\user\Neural_Network\Testing_data1'
s = valid_read(path)
for n in s:
    print(n)

C:\Users\user\Neural_Network\Testing_data1\0001.png
C:\Users\user\Neural_Network\Testing_data1\0002.png
C:\Users\user\Neural_Network\Testing_data1\0003.png
C:\Users\user\Neural_Network\Testing_data1\0004.png
C:\Users\user\Neural_Network\Testing_data1\0005.png
C:\Users\user\Neural_Network\Testing_data1\0006.png
C:\Users\user\Neural_Network\Testing_data1\0007.png
C:\Users\user\Neural_Network\Testing_data1\0008.png
C:\Users\user\Neural_Network\Testing_data1\0009.png
C:\Users\user\Neural_Network\Testing_data1\0010.png
C:\Users\user\Neural_Network\Testing_data1\0011.png
C:\Users\user\Neural_Network\Testing_data1\0012.png
C:\Users\user\Neural_Network\Testing_data1\0013.png
C:\Users\user\Neural_Network\Testing_data1\0014.png
C:\Users\user\Neural_Network\Testing_data1\0015.png
C:\Users\user\Neural_Network\Testing_data1\0016.png
C:\Users\user\Neural_Network\Testing_data1\0017.png
C:\Users\user\Neural_Network\Testing_data1\0018.png
C:\Users\user\Neural_Network\Testing_data1\0019.png
C:\Users\use

C:\Users\user\Neural_Network\Testing_data1\1312.png
C:\Users\user\Neural_Network\Testing_data1\1313.png
C:\Users\user\Neural_Network\Testing_data1\1314.png
C:\Users\user\Neural_Network\Testing_data1\1315.png
C:\Users\user\Neural_Network\Testing_data1\1316.png
C:\Users\user\Neural_Network\Testing_data1\1317.png
C:\Users\user\Neural_Network\Testing_data1\1318.png
C:\Users\user\Neural_Network\Testing_data1\1319.png
C:\Users\user\Neural_Network\Testing_data1\1320.png
C:\Users\user\Neural_Network\Testing_data1\1321.png
C:\Users\user\Neural_Network\Testing_data1\1322.png
C:\Users\user\Neural_Network\Testing_data1\1323.png
C:\Users\user\Neural_Network\Testing_data1\1324.png
C:\Users\user\Neural_Network\Testing_data1\1325.png
C:\Users\user\Neural_Network\Testing_data1\1326.png
C:\Users\user\Neural_Network\Testing_data1\1327.png
C:\Users\user\Neural_Network\Testing_data1\1328.png
C:\Users\user\Neural_Network\Testing_data1\1329.png
C:\Users\user\Neural_Network\Testing_data1\1330.png
C:\Users\use

C:\Users\user\Neural_Network\Testing_data1\2169.png
C:\Users\user\Neural_Network\Testing_data1\2170.png
C:\Users\user\Neural_Network\Testing_data1\2171.png
C:\Users\user\Neural_Network\Testing_data1\2172.png
C:\Users\user\Neural_Network\Testing_data1\2173.png
C:\Users\user\Neural_Network\Testing_data1\2174.png
C:\Users\user\Neural_Network\Testing_data1\2175.png
C:\Users\user\Neural_Network\Testing_data1\2176.png
C:\Users\user\Neural_Network\Testing_data1\2177.png
C:\Users\user\Neural_Network\Testing_data1\2178.png
C:\Users\user\Neural_Network\Testing_data1\2179.png
C:\Users\user\Neural_Network\Testing_data1\2180.png
C:\Users\user\Neural_Network\Testing_data1\2181.png
C:\Users\user\Neural_Network\Testing_data1\2182.png
C:\Users\user\Neural_Network\Testing_data1\2183.png
C:\Users\user\Neural_Network\Testing_data1\2184.png
C:\Users\user\Neural_Network\Testing_data1\2185.png
C:\Users\user\Neural_Network\Testing_data1\2186.png
C:\Users\user\Neural_Network\Testing_data1\2187.png
C:\Users\use

C:\Users\user\Neural_Network\Testing_data1\3226.png
C:\Users\user\Neural_Network\Testing_data1\3227.png
C:\Users\user\Neural_Network\Testing_data1\3228.png
C:\Users\user\Neural_Network\Testing_data1\3229.png
C:\Users\user\Neural_Network\Testing_data1\3230.png
C:\Users\user\Neural_Network\Testing_data1\3231.png
C:\Users\user\Neural_Network\Testing_data1\3232.png
C:\Users\user\Neural_Network\Testing_data1\3233.png
C:\Users\user\Neural_Network\Testing_data1\3234.png
C:\Users\user\Neural_Network\Testing_data1\3235.png
C:\Users\user\Neural_Network\Testing_data1\3236.png
C:\Users\user\Neural_Network\Testing_data1\3237.png
C:\Users\user\Neural_Network\Testing_data1\3238.png
C:\Users\user\Neural_Network\Testing_data1\3239.png
C:\Users\user\Neural_Network\Testing_data1\3240.png
C:\Users\user\Neural_Network\Testing_data1\3241.png
C:\Users\user\Neural_Network\Testing_data1\3242.png
C:\Users\user\Neural_Network\Testing_data1\3243.png
C:\Users\user\Neural_Network\Testing_data1\3244.png
C:\Users\use

C:\Users\user\Neural_Network\Testing_data1\4227.png
C:\Users\user\Neural_Network\Testing_data1\4228.png
C:\Users\user\Neural_Network\Testing_data1\4229.png
C:\Users\user\Neural_Network\Testing_data1\4230.png
C:\Users\user\Neural_Network\Testing_data1\4231.png
C:\Users\user\Neural_Network\Testing_data1\4232.png
C:\Users\user\Neural_Network\Testing_data1\4233.png
C:\Users\user\Neural_Network\Testing_data1\4234.png
C:\Users\user\Neural_Network\Testing_data1\4235.png
C:\Users\user\Neural_Network\Testing_data1\4236.png
C:\Users\user\Neural_Network\Testing_data1\4237.png
C:\Users\user\Neural_Network\Testing_data1\4238.png
C:\Users\user\Neural_Network\Testing_data1\4239.png
C:\Users\user\Neural_Network\Testing_data1\4240.png
C:\Users\user\Neural_Network\Testing_data1\4241.png
C:\Users\user\Neural_Network\Testing_data1\4242.png
C:\Users\user\Neural_Network\Testing_data1\4243.png
C:\Users\user\Neural_Network\Testing_data1\4244.png
C:\Users\user\Neural_Network\Testing_data1\4245.png
C:\Users\use

In [8]:
ts_w = model.test(s)

-執行開始

-執行結束

